In [1]:
# Imports
import pandas as pd
import geopandas as gpd
import numpy as np
from collections import Counter
import networkx as nx
import igraph as ig
import osmnx as ox
import h3
import shapely.wkt

In [2]:
from shapely.geometry import Point

In [3]:
from scipy.spatial import cKDTree

In [4]:
df = pd.read_csv(r'D:\bike\data\variables\hex_center.csv').drop(columns = 'Unnamed: 0')

In [5]:
geometry = df['geometry'].map(shapely.wkt.loads)
df = df.drop('geometry', axis=1)
gdf = gpd.GeoDataFrame(df, geometry=geometry)

In [6]:
gdf = gdf.set_crs("EPSG:4326")

In [7]:
gdf

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry
0,88411c0001fffff,4750.600957,39.216667,POINT (114.23688 22.75576)
1,88411c0009fffff,376.831173,30.344444,POINT (114.24023 22.74789)
2,88411c0011fffff,333.993458,3.900000,POINT (114.21419 22.75279)
3,88411c0019fffff,296.769036,7.794444,POINT (114.21754 22.74492)
4,88411c001dfffff,918.903705,9.866667,POINT (114.22287 22.75115)
...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (114.13048 22.53672)
1689,88411cc6c3fffff,600.830531,8.806667,POINT (114.47226 22.59562)
1690,88411cc6c7fffff,597.171997,4.483333,POINT (114.47758 22.60185)
1691,88411cc6cbfffff,326.080286,5.875000,POINT (114.47559 22.58775)


In [8]:
def distance_cbd(gdf, gdf_loc):
    """  
    Returns a DataFrame with an additional line that contains the distance to a given point
    
    Calculates the following:
        
        Features:
        ---------
        - Distance to CBD
 
    Args:
        - gdf: geodataframe with trip origin waypoint
        - gdf_loc: location of Point of Interest (format: shapely.geometry.point.Point)  
    Returns:
        - gdf: a DataFrame of shape (number of columns(gdf)+1, len_df) with the 
          computed features
    Last update: 2/12/21. By Felix.
    """
    
    # create numpy array
    np_geom = gdf.geometry.values
    # 1.create new column in dataframe to assign distance to CBD array to
    gdf['feature_distance_cbd'] = np_geom[:].distance(gdf_loc.geometry.iloc[0])
   
    return gdf

# distance_main_cbd

深圳中心坐标 Point(114.05266840592046,22.536904009346095)

In [9]:
point = Point(114.05266840592046,22.536904009346095)

In [10]:
points = gpd.GeoSeries(point)

In [11]:
gdf_loc = gpd.GeoDataFrame({'geometry':points})

In [12]:
gdf_loc = gdf_loc.set_crs('EPSG:4326')

In [13]:
gdf_loc = gdf_loc.to_crs('EPSG:32649')

In [14]:
gdf_loc

,geometry
0,POINT (814004.768 2495464.846)


In [15]:
gdf = gdf.to_crs('EPSG:32649')

In [16]:
distance_cbd(gdf, gdf_loc)

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry,feature_distance_cbd
0,88411c0001fffff,4750.600957,39.216667,POINT (832438.822 2520117.359),30782.475801
1,88411c0009fffff,376.831173,30.344444,POINT (832802.088 2519253.485),30318.948721
2,88411c0011fffff,333.993458,3.900000,POINT (830114.324 2519737.287),29131.928188
3,88411c0019fffff,296.769036,7.794444,POINT (830477.600 2518873.310),28623.599413
4,88411c001dfffff,918.903705,9.866667,POINT (831010.310 2519576.009),29504.858912
...,...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (822014.452 2495610.138),8011.001950
1689,88411cc6c3fffff,600.830531,8.806667,POINT (857048.700 2502916.554),43684.185658
1690,88411cc6c7fffff,597.171997,4.483333,POINT (857579.827 2503619.606),44331.545061
1691,88411cc6cbfffff,326.080286,5.875000,POINT (857411.966 2502052.818),43904.284972


In [17]:
gdf = gdf.rename(columns={'feature_distance_cbd':'distance_main_cbd'})

In [18]:
gdf

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry,distance_main_cbd
0,88411c0001fffff,4750.600957,39.216667,POINT (832438.822 2520117.359),30782.475801
1,88411c0009fffff,376.831173,30.344444,POINT (832802.088 2519253.485),30318.948721
2,88411c0011fffff,333.993458,3.900000,POINT (830114.324 2519737.287),29131.928188
3,88411c0019fffff,296.769036,7.794444,POINT (830477.600 2518873.310),28623.599413
4,88411c001dfffff,918.903705,9.866667,POINT (831010.310 2519576.009),29504.858912
...,...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (822014.452 2495610.138),8011.001950
1689,88411cc6c3fffff,600.830531,8.806667,POINT (857048.700 2502916.554),43684.185658
1690,88411cc6c7fffff,597.171997,4.483333,POINT (857579.827 2503619.606),44331.545061
1691,88411cc6cbfffff,326.080286,5.875000,POINT (857411.966 2502052.818),43904.284972


In [20]:
def get_shortest_dist(graph_ig, osmids, orig_osmid, dest_osmid, weight='length'):
    # calculate shortest distance using igraph
    return graph_ig.shortest_paths(
        source=osmids.index(orig_osmid),
        target=osmids.index(dest_osmid),
        weights=weight)[0][0]



def nearest_neighbour(gdA, gdB):
    """
    Function to calculate for every entry in gdA, the nearest neighbour
    among the points in gdB

    taken from https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe

    Args:
    - gdA: geodataframe with points in geometry column
    - gdB: geodataframe with points in geometry column

    Returns:
        - gdf_out: geodataframe wich is gdA + 2 columns containing
        the name of the closest point and the distance

    Last update: 13/04/21. By Felix.
    """
    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=False)
    gdf_out = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='distance')
        ],
        axis=1)
     
    return gdf_out


def convert_to_igraph(graph_nx, weight='length'):
    """
    Function to convert networkx (or osmnx) graph element to igraph

    Args:
    - graph_nx (networkx graph): multigraph object
    - weight (string) = 'length': attribute of the graph

    Returns:
        - G_ig (igraph element): converted graph
        - osmids (list): list with osm IDs of nodes

    Last update: 29/06/21. By Felix.
    """
    # retrieve list of osmid id's and relabel
    G_nx = graph_nx
    osmids = list(G_nx.nodes)
    G_nx = nx.relabel.convert_node_labels_to_integers(G_nx)
    # give each node its original osmid as attribute since we relabeled them
    osmid_values = {k: v for k, v in zip(G_nx.nodes, osmids)}
    nx.set_node_attributes(G_nx, osmid_values, "osmid")
    # convert networkx graph to igraph
    G_ig = ig.Graph(directed=True)
    G_ig.add_vertices(G_nx.nodes)
    G_ig.add_edges(G_nx.edges())
    G_ig.vs["osmid"] = osmids
    G_ig.es[weight] = list(nx.get_edge_attributes(G_nx, weight).values())
    return G_ig, osmids



In [21]:
def distance_cbd_shortest_dist(gdf, gdf_loc, graph):
    """  
    Returns a DataFrame with an additional line that contains the distance to a given point
    based on the shortest path calculated with igraph's shortest_path function.
    We convert to igraph in order to save 100ms per shortest_path calculation.
    For more info refer to the notebook shortest_path.ipynb or
    https://github.com/gboeing/osmnx-examples/blob/main/notebooks/14-osmnx-to-igraph.ipynb 
    
    Calculates the following:
        
        Features:
        ---------
        - Distance to CBD (based on graph network)
 
    Args:
        - gdf: geodataframe with trip origin waypoint
        - gdf_loc: location of Point of Interest (format: shapely.geometry.point.Point)
        - graph: Multigraph Object downloaded from osm  
    Returns:
        - gdf: a DataFrame of shape (number of columns(gdf)+1, len_gdf) with the 
          computed features
    Last update: 29/06/21. By Felix.
    """
    # then we have to convert the multigraph object to a dataframe
    gdf_nodes_4326, gdf_edges_4326 = ox.utils_graph.graph_to_gdfs(graph)
    
    gdf_4326 = gdf.to_crs(4326)
    gdf_loc_4326 = gdf_loc.to_crs(4326)

    # call nearest neighbour function
    gdf_orig_4326 = nearest_neighbour(gdf_4326, gdf_nodes_4326)
    gdf_dest_4326  = nearest_neighbour(gdf_loc_4326, gdf_nodes_4326)

    graph_ig, list_osmids = convert_to_igraph(graph)
    gdf['feature_distance_cbd'] = gdf_orig_4326.apply(lambda x: get_shortest_dist(graph_ig,
                                                                                     list_osmids, 
                                                                                     x.osmid, 
                                                                                     gdf_dest_4326.osmid.iloc[0], 
                                                                                     'length'),
                                                                                     axis=1)
    
    # add distance from hex center to nearest node (only for nodes where distance != inf)
    dist_start = gdf_orig_4326['distance'][gdf.feature_distance_cbd != np.inf]
    dist_end = gdf_dest_4326['distance'][0]
    gdf.feature_distance_cbd[gdf.feature_distance_cbd != np.inf] += dist_start + dist_end

    # check for nodes that could not be connected
    # create numpy array 
    np_geom = gdf.geometry[gdf.feature_distance_cbd == np.inf].values
    #assign distance to cbd array
    gdf.feature_distance_cbd[gdf.feature_distance_cbd == np.inf] = np_geom[:].distance(gdf_loc.geometry.iloc[0])

    print('Calculated distance to cbd based on shortest path')
    return gdf  



In [22]:
GHP = ox.graph_from_place("Shenzhen, Guangdong, China", network_type="bike")

In [23]:
gdf1 = distance_cbd_shortest_dist(gdf, gdf_loc, GHP)

C:\Users\jiawe\AppData\Local\Temp\ipykernel_12400\1926279704.py:3: DeprecationWarning: Graph.shortest_paths() is deprecated; use Graph.distances() instead
  return graph_ig.shortest_paths(


Calculated distance to cbd based on shortest path


C:\Users\jiawe\AppData\Local\Temp\ipykernel_12400\4048985023.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf.feature_distance_cbd[gdf.feature_distance_cbd != np.inf] += dist_start + dist_end
C:\Users\jiawe\AppData\Local\Temp\ipykernel_12400\4048985023.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf.feature_distance_cbd[gdf.feature_distance_cbd == np.inf] = np_geom[:].distance(gdf_loc.geometry.iloc[0])


In [24]:
gdf1

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry,distance_main_cbd,feature_distance_cbd
0,88411c0001fffff,4750.600957,39.216667,POINT (832438.822 2520117.359),30782.475801,36153.024028
1,88411c0009fffff,376.831173,30.344444,POINT (832802.088 2519253.485),30318.948721,35556.723607
2,88411c0011fffff,333.993458,3.900000,POINT (830114.324 2519737.287),29131.928188,38275.244399
3,88411c0019fffff,296.769036,7.794444,POINT (830477.600 2518873.310),28623.599413,36723.397263
4,88411c001dfffff,918.903705,9.866667,POINT (831010.310 2519576.009),29504.858912,36719.373613
...,...,...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (822014.452 2495610.138),8011.001950,8011.001950
1689,88411cc6c3fffff,600.830531,8.806667,POINT (857048.700 2502916.554),43684.185658,59345.427737
1690,88411cc6c7fffff,597.171997,4.483333,POINT (857579.827 2503619.606),44331.545061,59866.658630
1691,88411cc6cbfffff,326.080286,5.875000,POINT (857411.966 2502052.818),43904.284972,60316.100369


In [25]:
gdf1.to_csv(r'D:\bike\data\variables\distance_main_cbd.csv')

# distance_local_cbd

In [38]:
def get_shortest_dist(graph_ig, osmids, orig_osmid, dest_osmid, weight='length'):
    # calculate shortest distance using igraph
    return graph_ig.shortest_paths(
        source=osmids.index(orig_osmid),
        target=osmids.index(dest_osmid),
        weights=weight)[0][0]



def nearest_neighbour(gdA, gdB):
    """
    Function to calculate for every entry in gdA, the nearest neighbour
    among the points in gdB

    taken from https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe

    Args:
    - gdA: geodataframe with points in geometry column
    - gdB: geodataframe with points in geometry column

    Returns:
        - gdf_out: geodataframe wich is gdA + 2 columns containing
        the name of the closest point and the distance

    Last update: 13/04/21. By Felix.
    """
    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=False)
    gdf_out = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='distance')
        ],
        axis=1)
     
    return gdf_out


def convert_to_igraph(graph_nx, weight='length'):
    """
    Function to convert networkx (or osmnx) graph element to igraph

    Args:
    - graph_nx (networkx graph): multigraph object
    - weight (string) = 'length': attribute of the graph

    Returns:
        - G_ig (igraph element): converted graph
        - osmids (list): list with osm IDs of nodes

    Last update: 29/06/21. By Felix.
    """
    # retrieve list of osmid id's and relabel
    G_nx = graph_nx
    osmids = list(G_nx.nodes)
    G_nx = nx.relabel.convert_node_labels_to_integers(G_nx)
    # give each node its original osmid as attribute since we relabeled them
    osmid_values = {k: v for k, v in zip(G_nx.nodes, osmids)}
    nx.set_node_attributes(G_nx, osmid_values, "osmid")
    # convert networkx graph to igraph
    G_ig = ig.Graph(directed=True)
    G_ig.add_vertices(G_nx.nodes)
    G_ig.add_edges(G_nx.edges())
    G_ig.vs["osmid"] = osmids
    G_ig.es[weight] = list(nx.get_edge_attributes(G_nx, weight).values())
    return G_ig, osmids



In [65]:
def distance_local_cbd(gdf, gdf_loc_local):
    """
    Function to caluclate location of closest local city center for each point. 
    
    Args:
    - gdf: geodataframe with points in geometry column
    - gdf_loc_local: geodataframe with points in geometry column
    Returns:
        - gdf_out: geodataframe with trips only on either weekdays or weekends
    Last update: 13/04/21. By Felix.
    """  
    # call nearest neighbour function
    gdf_out = nearest_neighbour(gdf, gdf_loc_local)
    # rename columns and drop unneccessary ones
    gdf_out = gdf_out.rename(columns={"distance": "distance_local_cbd"})
    return gdf_out


def distance_local_cbd_shortest_dist(gdf, gdf_loc_local, graph):
    """  
    Returns a DataFrame with an additional line that contains the distance to points in gdf_loc_local
    based on the shortest path calculated with igraph's shortest_path function.
    We convert to igraph in order to save 100ms per shortest_path calculation.
    For more info refer to the notebook shortest_path.ipynb or
    https://github.com/gboeing/osmnx-examples/blob/main/notebooks/14-osmnx-to-igraph.ipynb 
    Calculates the following:
        
        Features:
        ---------
        - Distance to local cbd (based on graph network)
    Args:
        - gdf: geodataframe with trip origin waypoint
        - gdf_loc: location of Points of Interest (format: shapely.geometry.point.Point)
        - graph: Multigraph Object downloaded from osm  
    Returns:
        - gdf: a DataFrame of shape (number of columns(gdf)+1, len_gdf) with the 
            computed features
    Last update: 01/07/21. By Felix.
    """


    # call nearest neighbour to find nearest local center
    gdf_out = nearest_neighbour(gdf, gdf_loc_local)
    # rename distance column
    gdf_out = gdf_out.rename(columns={'distance':'distance_crow'})
    # remove unnecessary columns

    # convert input gdf to crs
    gdf_4326 = gdf_out.to_crs(4326)
    gdf_loc_local_4326 = gdf_loc_local.to_crs(4326)

    # then we have to convert the multigraph object to a dataframe
    gdf_nodes_4326, gdf_edges_4326 = ox.utils_graph.graph_to_gdfs(graph)
    # call nearest neighbour function to find nearest node
    gdf_orig_4326 = nearest_neighbour(gdf_4326, gdf_nodes_4326)
    gdf_dest_4326  = nearest_neighbour(gdf_loc_local_4326, gdf_nodes_4326)

    # merge on node ID 
    gdf_merge_4326 =  gdf_orig_4326.merge(gdf_dest_4326,how='left',on=['osmid'])

    # convert to igraph
    graph_ig, list_osmids = convert_to_igraph(graph)
    
    # call get shortest dist func, where gdf_merge_3426.osmid_x is nearest node from starting point and osmid_y is 
    # nearest node from end destination (one of the neighbourhood centers)
    gdf['feature_distance_local_cbd'] = gdf_merge_4326.apply(lambda x: get_shortest_dist(graph_ig,
                                                                                    list_osmids, 
                                                                                     x.osmid,
                                                                                      gdf_dest_4326.osmid.iloc[0],
                                                                                    'length'),
                                                                                    axis=1)

    # add distance from hex center to nearest node (only for nodes where distance != inf)
    dist_start = gdf_orig_4326['distance'][gdf.feature_distance_local_cbd != np.inf]
    dist_end = gdf_dest_4326['distance'][0]
    gdf.feature_distance_local_cbd[gdf.feature_distance_local_cbd != np.inf] += dist_start + dist_end


    # check for nodes that could not be connected
    # create numpy array 
    np_geom = gdf.geometry[gdf.feature_distance_local_cbd == np.inf].values
    #assign distance to cbd array
    gdf.feature_distance_local_cbd[gdf.feature_distance_local_cbd == np.inf] = np_geom[:].distance(gdf_loc.geometry.iloc[0])
    
    
    print('Calculated distance to local cbd based on shortest path')
    return gdf 

In [96]:
df = pd.read_csv(r'D:\bike\data\variables\hex_center.csv').drop(columns = 'Unnamed: 0')
geometry = df['geometry'].map(shapely.wkt.loads)
df = df.drop('geometry', axis=1)
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf = gdf.set_crs("EPSG:4326")
gdf_1 = gdf.to_crs('EPSG:32649')
gdf_1

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry
0,88411c0001fffff,4750.600957,39.216667,POINT (832438.822 2520117.359)
1,88411c0009fffff,376.831173,30.344444,POINT (832802.088 2519253.485)
2,88411c0011fffff,333.993458,3.900000,POINT (830114.324 2519737.287)
3,88411c0019fffff,296.769036,7.794444,POINT (830477.600 2518873.310)
4,88411c001dfffff,918.903705,9.866667,POINT (831010.310 2519576.009)
...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (822014.452 2495610.138)
1689,88411cc6c3fffff,600.830531,8.806667,POINT (857048.700 2502916.554)
1690,88411cc6c7fffff,597.171997,4.483333,POINT (857579.827 2503619.606)
1691,88411cc6cbfffff,326.080286,5.875000,POINT (857411.966 2502052.818)


In [97]:
df = pd.read_csv(r'D:\bike\data\variables\gdf_local_cbd\gdf_local_cbd_shpping.csv').drop(columns = 'Unnamed: 0')

geometry = df['geometry'].map(shapely.wkt.loads)
df = df.drop('geometry', axis=1)
gdf_loc_local = gpd.GeoDataFrame(df, geometry=geometry)

gdf_loc_local = gdf_loc_local.set_crs("EPSG:4326")
gdf_loc_local_1 = gdf_loc_local.to_crs("EPSG:32649")

In [98]:
result1 = distance_local_cbd_shortest_dist(gdf_1, gdf_loc_local_1, GHP)

C:\Users\jiawe\AppData\Local\Temp\ipykernel_12400\1926279704.py:3: DeprecationWarning: Graph.shortest_paths() is deprecated; use Graph.distances() instead
  return graph_ig.shortest_paths(


Calculated distance to local cbd based on shortest path


C:\Users\jiawe\AppData\Local\Temp\ipykernel_12400\2553119468.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf.feature_distance_local_cbd[gdf.feature_distance_local_cbd != np.inf] += dist_start + dist_end
C:\Users\jiawe\AppData\Local\Temp\ipykernel_12400\2553119468.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf.feature_distance_local_cbd[gdf.feature_distance_local_cbd == np.inf] = np_geom[:].distance(gdf_loc.geometry.iloc[0])


In [99]:
result1

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry,feature_distance_local_cbd
0,88411c0001fffff,4750.600957,39.216667,POINT (832438.822 2520117.359),50569.743407
1,88411c0009fffff,376.831173,30.344444,POINT (832802.088 2519253.485),49973.442987
2,88411c0011fffff,333.993458,3.900000,POINT (830114.324 2519737.287),52691.963779
3,88411c0019fffff,296.769036,7.794444,POINT (830477.600 2518873.310),51140.116643
4,88411c001dfffff,918.903705,9.866667,POINT (831010.310 2519576.009),51136.092993
...,...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (822014.452 2495610.138),8011.001950
1689,88411cc6c3fffff,600.830531,8.806667,POINT (857048.700 2502916.554),73762.147117
1690,88411cc6c7fffff,597.171997,4.483333,POINT (857579.827 2503619.606),74283.378010
1691,88411cc6cbfffff,326.080286,5.875000,POINT (857411.966 2502052.818),74732.819749


In [100]:
result2 = distance_local_cbd(gdf_1, gdf_loc_local_1)

In [101]:
result2

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry,feature_distance_local_cbd,index,distance_local_cbd
0,88411c0001fffff,4750.600957,39.216667,POINT (832438.822 2520117.359),50569.743407,9,3749.664803
1,88411c0009fffff,376.831173,30.344444,POINT (832802.088 2519253.485),49973.442987,9,2829.151062
2,88411c0011fffff,333.993458,3.900000,POINT (830114.324 2519737.287),52691.963779,9,5145.809528
3,88411c0019fffff,296.769036,7.794444,POINT (830477.600 2518873.310),51140.116643,9,4403.502578
4,88411c001dfffff,918.903705,9.866667,POINT (831010.310 2519576.009),51136.092993,9,4320.077572
...,...,...,...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (822014.452 2495610.138),8011.001950,2,4586.849646
1689,88411cc6c3fffff,600.830531,8.806667,POINT (857048.700 2502916.554),73762.147117,9,26610.217453
1690,88411cc6c7fffff,597.171997,4.483333,POINT (857579.827 2503619.606),74283.378010,9,26705.373859
1691,88411cc6cbfffff,326.080286,5.875000,POINT (857411.966 2502052.818),74732.819749,9,27379.594196


In [102]:
result2.describe()

,Bike_Travel_Distance,BIKE_MINU,feature_distance_local_cbd,index,distance_local_cbd
count,1693.000000,1693.000000,1693.000000,1693.000000,1693.000000
mean,1858.925230,15.672839,26958.383804,4.203780,5501.250383
std,1751.255947,9.596386,13175.374811,2.886413,3480.739094
min,0.000000,0.333333,1048.694365,0.000000,67.197882
25%,1064.878968,10.993750,17591.924021,2.000000,3205.796672
50%,1332.179167,13.253858,27329.865252,4.000000,4988.669808
75%,1843.480801,16.662745,34505.259938,7.000000,7168.324278
max,14917.186670,104.633333,91909.477490,9.000000,40327.817188


In [103]:
result2.to_csv(r'D:\bike\data\variables\gdf_local_cbd\distance_local_cbd_shpping.csv')

# distance_bus_station


In [104]:
df = pd.read_csv(r'D:\bike\data\variables\hex_center.csv').drop(columns = 'Unnamed: 0')
geometry = df['geometry'].map(shapely.wkt.loads)
df = df.drop('geometry', axis=1)
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf = gdf.set_crs("EPSG:4326")
gdf_1 = gdf.to_crs('EPSG:32649')
gdf_1

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry
0,88411c0001fffff,4750.600957,39.216667,POINT (832438.822 2520117.359)
1,88411c0009fffff,376.831173,30.344444,POINT (832802.088 2519253.485)
2,88411c0011fffff,333.993458,3.900000,POINT (830114.324 2519737.287)
3,88411c0019fffff,296.769036,7.794444,POINT (830477.600 2518873.310)
4,88411c001dfffff,918.903705,9.866667,POINT (831010.310 2519576.009)
...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (822014.452 2495610.138)
1689,88411cc6c3fffff,600.830531,8.806667,POINT (857048.700 2502916.554)
1690,88411cc6c7fffff,597.171997,4.483333,POINT (857579.827 2503619.606)
1691,88411cc6cbfffff,326.080286,5.875000,POINT (857411.966 2502052.818)


In [106]:
gdf_loc_local = gpd.read_file(r"D:\bike\data\bus_station_shp\bus_station_point.shp")

gdf_loc_local = gdf_loc_local.set_crs("EPSG:4326")
gdf_loc_local_1 = gdf_loc_local.to_crs("EPSG:32649")

In [107]:
gdf_loc_local_1

,match,id_station,location,name,sequence,id,busstops,lng,lat,geometry
0,0,BV10242435,"114.118955,22.531607",火车站,1,440300014163,"[{'id': 'BV10242435', 'location': '114.118955,...",114.118955,22.531607,POINT (820840.162 2495018.869)
1,1,BV11042896,"114.118993,22.534839",人民南地铁站,2,440300014163,"[{'id': 'BV10242435', 'location': '114.118955,...",114.118993,22.534839,POINT (820836.595 2495377.088)
2,2,BV11161011,"114.118797,22.537426",罗湖小学2,3,440300014163,"[{'id': 'BV10242435', 'location': '114.118955,...",114.118797,22.537426,POINT (820810.433 2495663.332)
3,3,BV11354437,"114.118689,22.540595",国贸1,4,440300014163,"[{'id': 'BV10242435', 'location': '114.118955,...",114.118689,22.540595,POINT (820791.982 2496014.257)
4,4,BV10382979,"114.121866,22.547661",东门3,5,440300014163,"[{'id': 'BV10242435', 'location': '114.118955,...",114.121866,22.547661,POINT (821102.628 2496804.076)
...,...,...,...,...,...,...,...,...,...,...
26401,7,BV10245462,"113.962624,22.5196",滨海沙河东立交,8,440300065771,"[{'id': 'BV10243592', 'location': '113.93169,2...",113.962624,22.5196,POINT (804774.642 2493360.943)
26402,8,BV10245463,"113.98391,22.519501",滨海深湾立交,9,440300065771,"[{'id': 'BV10243592', 'location': '113.93169,2...",113.98391,22.519501,POINT (806966.024 2493393.563)
26403,9,BV11451848,"113.995796,22.522141",深圳湾公园地铁站,10,440300065771,"[{'id': 'BV10243592', 'location': '113.93169,2...",113.995796,22.522141,POINT (808183.709 2493710.553)
26404,10,BV10244085,"113.999962,22.524563",红树林,11,440300065771,"[{'id': 'BV10243592', 'location': '113.93169,2...",113.999962,22.524563,POINT (808607.165 2493987.516)


In [108]:
result1 = distance_local_cbd_shortest_dist(gdf_1, gdf_loc_local_1, GHP)

C:\Users\jiawe\AppData\Local\Temp\ipykernel_12400\1926279704.py:3: DeprecationWarning: Graph.shortest_paths() is deprecated; use Graph.distances() instead
  return graph_ig.shortest_paths(


Calculated distance to local cbd based on shortest path


C:\Users\jiawe\AppData\Local\Temp\ipykernel_12400\2553119468.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf.feature_distance_local_cbd[gdf.feature_distance_local_cbd != np.inf] += dist_start + dist_end
C:\Users\jiawe\AppData\Local\Temp\ipykernel_12400\2553119468.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf.feature_distance_local_cbd[gdf.feature_distance_local_cbd == np.inf] = np_geom[:].distance(gdf_loc.geometry.iloc[0])


In [109]:
result1

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry,feature_distance_local_cbd
0,88411c0001fffff,4750.600957,39.216667,POINT (832438.822 2520117.359),31485.330475
1,88411c0009fffff,376.831173,30.344444,POINT (832802.088 2519253.485),30889.030055
2,88411c0011fffff,333.993458,3.900000,POINT (830114.324 2519737.287),33607.550847
3,88411c0019fffff,296.769036,7.794444,POINT (830477.600 2518873.310),32055.703710
4,88411c001dfffff,918.903705,9.866667,POINT (831010.310 2519576.009),32051.680060
...,...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (822014.452 2495610.138),45607.496568
1689,88411cc6c3fffff,600.830531,8.806667,POINT (857048.700 2502916.554),45314.706184
1690,88411cc6c7fffff,597.171997,4.483333,POINT (857579.827 2503619.606),45848.998077
1691,88411cc6cbfffff,326.080286,5.875000,POINT (857411.966 2502052.818),45848.998816


In [110]:
result2 = distance_local_cbd(gdf_1, gdf_loc_local_1)

In [111]:
result2

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry,feature_distance_local_cbd,index,match,id_station,location,name,sequence,id,busstops,lng,lat,distance_local_cbd
0,88411c0001fffff,4750.600957,39.216667,POINT (832438.822 2520117.359),31485.330475,16410,20,BV11397759,"114.234089,22.751897",承翰陶源,21,900000030235,"[{'id': 'BV11239587', 'location': '114.225998,...",114.234089,22.751897,515.001534
1,88411c0009fffff,376.831173,30.344444,POINT (832802.088 2519253.485),30889.030055,12034,5,BV10625492,"114.240964,22.747114",龙西五联路,6,440300015497,"[{'id': 'BV11049037', 'location': '114.245543,...",114.240964,22.747114,114.885170
2,88411c0011fffff,333.993458,3.900000,POINT (830114.324 2519737.287),33607.550847,10922,44,BV11154408,"114.217444,22.748171",鸿威的森林,45,440300015437,"[{'id': 'BV10388274', 'location': '114.348234,...",114.217444,22.748171,611.020238
3,88411c0019fffff,296.769036,7.794444,POINT (830477.600 2518873.310),32055.703710,5065,0,BV11154408,"114.217407,22.748133",鸿威的森林,1,440300014633,"[{'id': 'BV11154408', 'location': '114.217407,...",114.217407,22.748133,356.260534
4,88411c001dfffff,918.903705,9.866667,POINT (831010.310 2519576.009),32051.680060,10921,43,BV10383488,"114.221214,22.747969",五联利源手袋厂,44,440300015437,"[{'id': 'BV10388274', 'location': '114.348234,...",114.221214,22.747969,392.100246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (822014.452 2495610.138),45607.496568,17165,29,BV10246395,"114.130056,22.538906",文锦渡客运站南,30,900000041610,"[{'id': 'BV11143658', 'location': '114.189384,...",114.130056,22.538906,246.264865
1689,88411cc6c3fffff,600.830531,8.806667,POINT (857048.700 2502916.554),45314.706184,6374,0,BV11633740,"114.473433,22.596319",大鹏总站,1,900000046221,"[{'id': 'BV11633740', 'location': '114.473433,...",114.473433,22.596319,143.521556
1690,88411cc6c7fffff,597.171997,4.483333,POINT (857579.827 2503619.606),45848.998077,26316,11,BV10619713,"114.478828,22.599262",大鹏山庄,12,900000164355,"[{'id': 'BV10569521', 'location': '114.516337,...",114.478828,22.599262,314.274930
1691,88411cc6cbfffff,326.080286,5.875000,POINT (857411.966 2502052.818),45848.998816,16761,18,BV10595785,"114.478828,22.587773",华兴新村,19,900000029927,"[{'id': 'BV10248359', 'location': '114.528442,...",114.478828,22.587773,333.194696


In [112]:
result2.describe()

,Bike_Travel_Distance,BIKE_MINU,feature_distance_local_cbd,index,match,sequence,id,distance_local_cbd
count,1693.000000,1693.000000,1693.000000,1693.000000,1693.000000,1693.000000,1.693000e+03,1693.000000
mean,1858.925230,15.672839,25829.551458,14204.262847,16.128175,17.128175,6.167944e+11,361.363182
std,1751.255947,9.596386,11656.716884,7497.531242,13.694557,13.694557,2.236376e+11,343.914144
min,0.000000,0.333333,1152.896558,18.000000,0.000000,1.000000,4.403000e+11,1.896040
25%,1064.878968,10.993750,17659.522858,8161.000000,5.000000,6.000000,4.403000e+11,135.339574
50%,1332.179167,13.253858,25395.798477,14742.000000,13.000000,14.000000,4.403000e+11,254.945389
75%,1843.480801,16.662745,32697.896951,20859.000000,24.000000,25.000000,9.000001e+11,462.108607
max,14917.186670,104.633333,55275.007034,26402.000000,77.000000,78.000000,9.000002e+11,2629.596813


In [113]:
result2.to_csv(r'D:\bike\data\variables\gdf_local_cbd\distance_local_cbd_bus.csv')

# rail_station

In [114]:
df = pd.read_csv(r'D:\bike\data\variables\hex_center.csv').drop(columns = 'Unnamed: 0')
geometry = df['geometry'].map(shapely.wkt.loads)
df = df.drop('geometry', axis=1)
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf = gdf.set_crs("EPSG:4326")
gdf_1 = gdf.to_crs('EPSG:32649')
gdf_1

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry
0,88411c0001fffff,4750.600957,39.216667,POINT (832438.822 2520117.359)
1,88411c0009fffff,376.831173,30.344444,POINT (832802.088 2519253.485)
2,88411c0011fffff,333.993458,3.900000,POINT (830114.324 2519737.287)
3,88411c0019fffff,296.769036,7.794444,POINT (830477.600 2518873.310)
4,88411c001dfffff,918.903705,9.866667,POINT (831010.310 2519576.009)
...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (822014.452 2495610.138)
1689,88411cc6c3fffff,600.830531,8.806667,POINT (857048.700 2502916.554)
1690,88411cc6c7fffff,597.171997,4.483333,POINT (857579.827 2503619.606)
1691,88411cc6cbfffff,326.080286,5.875000,POINT (857411.966 2502052.818)


In [115]:
gdf_loc_local = gpd.read_file(r"D:\bike\data\rail_station_point\rail_station_point.shp")

gdf_loc_local = gdf_loc_local.set_crs("EPSG:4326")
gdf_loc_local_1 = gdf_loc_local.to_crs("EPSG:32649")

In [116]:
gdf_loc_local_1

,linepoint,station,linename,lng,lat,line,id,geometry
0,0,罗湖,地铁1号线(罗宝线)(罗湖-机场东),114.113806,22.534290,地铁1号线,1,POINT (820303.907 2495305.059)
1,1,国贸,地铁1号线(罗宝线)(罗湖-机场东),114.113641,22.542407,地铁1号线,2,POINT (820268.176 2496204.245)
2,2,老街,地铁1号线(罗宝线)(罗湖-机场东),114.111150,22.546795,地铁1号线,3,POINT (820001.682 2496685.110)
3,3,大剧院,地铁1号线(罗宝线)(罗湖-机场东),114.103284,22.544879,地铁1号线,4,POINT (819196.413 2496455.916)
4,4,科学馆,地铁1号线(罗宝线)(罗湖-机场东),114.089758,22.543326,地铁1号线,5,POINT (817807.802 2496254.975)
...,...,...,...,...,...,...,...,...
709,0,会展城,地铁20号线(会展城-机场北),113.764604,22.714993,地铁20号线,1,POINT (783990.016 2514617.180)
710,1,国展北,地铁20号线(会展城-机场北),113.771784,22.706029,地铁20号线,2,POINT (784746.560 2513637.915)
711,2,国展,地铁20号线(会展城-机场北),113.772743,22.698833,地铁20号线,3,POINT (784860.025 2512842.467)
712,3,国展南,地铁20号线(会展城-机场北),113.773348,22.687057,地铁20号线,4,POINT (784946.565 2511539.032)


In [117]:
result1 = distance_local_cbd_shortest_dist(gdf_1, gdf_loc_local_1, GHP)

C:\Users\jiawe\AppData\Local\Temp\ipykernel_12400\1926279704.py:3: DeprecationWarning: Graph.shortest_paths() is deprecated; use Graph.distances() instead
  return graph_ig.shortest_paths(


Calculated distance to local cbd based on shortest path


C:\Users\jiawe\AppData\Local\Temp\ipykernel_12400\2553119468.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf.feature_distance_local_cbd[gdf.feature_distance_local_cbd != np.inf] += dist_start + dist_end
C:\Users\jiawe\AppData\Local\Temp\ipykernel_12400\2553119468.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf.feature_distance_local_cbd[gdf.feature_distance_local_cbd == np.inf] = np_geom[:].distance(gdf_loc.geometry.iloc[0])


In [118]:
result1

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry,feature_distance_local_cbd
0,88411c0001fffff,4750.600957,39.216667,POINT (832438.822 2520117.359),31447.343945
1,88411c0009fffff,376.831173,30.344444,POINT (832802.088 2519253.485),30851.043524
2,88411c0011fffff,333.993458,3.900000,POINT (830114.324 2519737.287),33569.564316
3,88411c0019fffff,296.769036,7.794444,POINT (830477.600 2518873.310),32017.717180
4,88411c001dfffff,918.903705,9.866667,POINT (831010.310 2519576.009),32013.693530
...,...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (822014.452 2495610.138),8011.001950
1689,88411cc6c3fffff,600.830531,8.806667,POINT (857048.700 2502916.554),43684.185658
1690,88411cc6c7fffff,597.171997,4.483333,POINT (857579.827 2503619.606),1815.683547
1691,88411cc6cbfffff,326.080286,5.875000,POINT (857411.966 2502052.818),1815.684286


In [119]:
result2 = distance_local_cbd(gdf_1, gdf_loc_local_1)

In [120]:
result2

,hex_id,Bike_Travel_Distance,BIKE_MINU,geometry,feature_distance_local_cbd,index,linepoint,station,linename,lng,lat,line,id,distance_local_cbd
0,88411c0001fffff,4750.600957,39.216667,POINT (832438.822 2520117.359),31447.343945,88,28,龙城广场,地铁3号线(龙岗线)(福保-双龙),114.249640,22.719795,地铁3号线,29,4196.610157
1,88411c0009fffff,376.831173,30.344444,POINT (832802.088 2519253.485),30851.043524,88,28,龙城广场,地铁3号线(龙岗线)(福保-双龙),114.249640,22.719795,地铁3号线,29,3261.504260
2,88411c0011fffff,333.993458,3.900000,POINT (830114.324 2519737.287),33569.564316,88,28,龙城广场,地铁3号线(龙岗线)(福保-双龙),114.249640,22.719795,地铁3号线,29,5162.983365
3,88411c0019fffff,296.769036,7.794444,POINT (830477.600 2518873.310),32017.717180,94,3,吉祥,地铁3号线(龙岗线)(双龙-福保),114.239574,22.712529,地铁3号线,4,4245.481681
4,88411c001dfffff,918.903705,9.866667,POINT (831010.310 2519576.009),32013.693530,88,28,龙城广场,地铁3号线(龙岗线)(福保-双龙),114.249640,22.719795,地铁3号线,29,4433.839199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1688,88411cb9e7fffff,774.091405,8.425977,POINT (822014.452 2495610.138),8011.001950,439,31,文锦,地铁9号线(梅林线)(前湾-文锦),114.125940,22.545132,地铁9号线,32,1042.960995
1689,88411cc6c3fffff,600.830531,8.806667,POINT (857048.700 2502916.554),43684.185658,681,13,坪山围,地铁14号线(东部快线)(岗厦北-沙田),114.338592,22.693865,地铁14号线,14,17544.965266
1690,88411cc6c7fffff,597.171997,4.483333,POINT (857579.827 2503619.606),1815.683547,681,13,坪山围,地铁14号线(东部快线)(岗厦北-沙田),114.338592,22.693865,地铁14号线,14,17567.204967
1691,88411cc6cbfffff,326.080286,5.875000,POINT (857411.966 2502052.818),1815.684286,681,13,坪山围,地铁14号线(东部快线)(岗厦北-沙田),114.338592,22.693865,地铁14号线,14,18361.418739


In [121]:
result2.describe()

,Bike_Travel_Distance,BIKE_MINU,feature_distance_local_cbd,index,linepoint,lng,lat,id,distance_local_cbd
count,1693.000000,1693.000000,1693.000000,1693.000000,1693.000000,1693.000000,1693.000000,1693.000000,1693.000000
mean,1858.925230,15.672839,27367.344212,379.012404,11.052569,114.011640,22.650822,12.052569,1927.852858
std,1751.255947,9.596386,12036.064221,211.601840,8.101029,0.148260,0.078480,8.101029,2518.726251
min,0.000000,0.333333,653.601737,0.000000,0.000000,113.764604,22.472414,1.000000,19.105623
25%,1064.878968,10.993750,18618.802173,185.000000,5.000000,113.892862,22.584693,6.000000,661.269666
50%,1332.179167,13.253858,26333.400656,370.000000,10.000000,113.988932,22.654319,11.000000,1175.760179
75%,1843.480801,16.662745,36421.044909,573.000000,15.000000,114.120903,22.706259,16.000000,2203.158275
max,14917.186670,104.633333,69157.508515,713.000000,32.000000,114.397234,22.807506,33.000000,31476.324989


In [122]:
result2.to_csv(r'D:\bike\data\variables\gdf_local_cbd\distance_local_cbd_rail_station.csv')